In [1]:
import numpy as np
from scipy.integrate import odeint
import paho.mqtt.client as mqtt
import time
import socket

In [2]:
def f(y, t, params):
    theta, omega = y      # unpack current values of y
    Q, d, Omega = params  # unpack parameters
    derivs = [omega,      # list of dy/dt=f functions
             -omega/Q + np.sin(theta) + d*np.cos(Omega*t)]
    return derivs

def solver():
    # Parameters
    Q = 2.0          # quality factor (inverse damping)
    d = 1.5          # forcing amplitude
    Omega = 0.65     # drive frequency

    # Initial values
    theta0 = 0.0     # initial angular displacement
    omega0 = 0.0     # initial angular velocity

    # Bundle parameters for ODE solver
    params = [Q, d, Omega]

    # Bundle initial conditions for ODE solver
    y0 = [theta0, omega0]

    # Make time array for solution
    tStop = 200.
    tInc = 0.05
    t = np.arange(0., tStop, tInc)
    
    cnt = 0
    
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print ('Socket created')

    s.connect((HOST, PORT))

    print('Points numbers: ' + str(len(t)))
    
    for x in range(len(t)-1):
        y = odeint(f, y0, [t[x], t[x+1]], args=(params,))
        y0 = y[1]
        cnt = cnt + 1
    
        if(cnt>= 10 ):
            s.send(y[0,0])
            cnt = 0;
        for i in range(100000):
            a = i
            
    s.close()
    print ('Socket closed')

In [ ]:
def on_message(client, userdata, message):
    modelo_str = str(message.payload.decode("utf-8"))
    print(modelo_str)
    print('Started a solution')
    solver()
    print('Solution complete')

broker_address="10.0.0.101"
client = mqtt.Client("P1") #create new instance
client.on_message=on_message #attach function to callback
client.connect(broker_address) #connect to broker
client.loop_start() #start the loop
client.subscribe("modelo/#")

# Configs for tcp socket
HOST = '10.0.0.101' # Symbolic name, meaning all available interfaces
PORT = 8880 #Arbitrary non-privileged port

print("Running")

while True:
    time.sleep(1)

Running
omega$-omega/Q + sen(theta) + d*cos(Omega*t)$Q = 10, d = 5, Omega = 1$tau = 0, omega = 0, t = 0
Started a solution
Socket created
Points numbers: 4000
Socket closed
Solution complete
